In [ ]:
!pip install snowflake-connector-python

In [1]:
cd '/Users/abdulrahmanmohsen/Documents/GitHub'

/Users/abdulrahmanmohsen/Documents/GitHub


/Users/abdulrahmanmohsen/opt/miniconda3/envs/ml/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import sys,os
import pandas as pd
from snowflake.connector import connect
sys.path.append('data_science_utils/')
import data_science_utils
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
from data_science_utils import getcode, run_sql_in_directory, get_connection, upload_large_table
os.getenv('SNOWFLAKE_CONNECTIONS_CONFIG')


'/Users/abdulrahmanmohsen/Documents/data_science_utils-main/sf_config.toml'

In [3]:
connection = get_connection(
    database='DATA_SCIENCE_DB'
, schema='PUBLIC'
, role='SCIENTIST'
, warehouse='DATA_SCIENCE_WH'
    
)

In [4]:
PROVIDER_RANKING_NPS = getcode("SELECT * FROM DATA_SCIENCE_DB.PUBLIC.PROVIDER_RANKING_FINAL", connection = connection)
NPS_DATA = getcode("SELECT * FROM DATA_SCIENCE_DB.PUBLIC.NPS_PROVIDER_RAW", connection = connection)

In [5]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli", device = 'mps')

In [6]:
NPS_DATA

,provider_type,encounter_date,provider_name,category,we_are_sorry_that_you_had_an_unpleasant_experience._can_you_please_provide_more_information_on_why_you_gave_that_rating?,manual_categorization,manual_sub-category,we_are_glad_you_had_a_great_experience._can_you_please_provide_more_information_on_why_you_gave_that_rating?,thank_you_for_your_response._can_you_please_provide_more_information_on_why_you_gave_that_rating?,facility_cleanliness,overall_wait_time,staff_attitude/conduct,consultation_quality,equipment_quality,medication_quality,approval_wait_time,overall_wait_time2
0,1,01 Aug 2023,REHOBOTH SPECIALIST HOSPITAL,Passive,None,None,None,None,It took time before attending with my condition,Satisfied,Neutral,Satisfied,Satisfied,Neutral,Satisfied,None,None
1,1,01 Aug 2023,Faith Care Hospital,Detractor,"I visited on the 31st of July, and was told to...",Bad customer service,None,None,None,Satisfied,Dissatisfied,Neutral,Neutral,Neutral,Dissatisfied,None,None
2,9,01 Aug 2023,THALIA BY ALICE,Promoter,None,Good customer service,Staff attitude,Very courteous staff,None,Satisfied,Satisfied,Satisfied,None,Satisfied,None,None,None
3,6,01 Aug 2023,Clina - Lancet Laboratories - Ikeja,Passive,None,None,None,None,Swift service,Satisfied,Neutral,Neutral,Satisfied,Satisfied,Satisfied,None,None
4,1,01 Aug 2023,GreenLife Hospital -Ilupeju,Promoter,None,Good customer service,Staff attitude,They handle customer with care n love and quic...,None,Satisfied,Satisfied,Satisfied,Satisfied,Neutral,Satisfied,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22947,1,12 Jan 2024,White Dove Hospital,Detractor,It was not a pleasant experience. It took thre...,None,None,None,None,Neutral,Satisfied,Dissatisfied,Neutral,Dissatisfied,Dissatisfied,Not Applicable,More than 30 mins but within 1 hour
22948,6,31 Jan 2024,EL-LAB LABORATORIES LIMITED - FESTAC,Promoter,None,None,None,Great customer service,None,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,15 mins or less,More than 30 mins but within 1 hour
22949,1,23 Jan 2024,Peamak Hospital and Maternity Centre,Passive,None,None,None,None,They have well trained medical practitioners,None,None,None,None,None,None,None,None
22950,1,31 Jan 2024,Greenlands Specialist Hospital,Promoter,None,None,None,I like the Greenland for the good effort on me,None,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,More than 15 mins but within 30 mins,More than 30 mins but within 1 hour


In [7]:
PROVIDER_RANKING_NPS

,provider_id,provider_name,type_service,type_service_name,provider_latitude,provider_longitude,is_internal_provider,address,telephone,provider_tier,...,raw_cost_star_rating,has_cost_stars,fwa_multiplier,months_since_last_fraud,cost_star_rating,clinical_stars,months_since_last_morbidity_incident,months_since_last_mortality_incident,latitude,longitude
0,5556,Bioflourishing Hospital,Medical services,Hospital,6.516740,3.307932,0,"63 BAYO OYEWALE,AGO PALACE WAY OKOTA","08037016681, 09018092810",4,...,2.500000,0,1.000000,NaN,2.500000,5.0,NaN,NaN,6.516740,3.307932
1,5554,Hephzibah Eye Care - Port Harcourt,Optical services,Optical Center,4.824801,6.988020,0,"Tombia Street, opposite Mandela Car Wash,GRA P...",07051590488,3,...,2.500000,0,1.000000,NaN,2.500000,5.0,NaN,NaN,4.824801,6.988020
2,5554,Hephzibah Eye Care - Port Harcourt,Optical services,Optical Center,4.824801,6.988020,0,"Tombia Street, opposite Mandela Car Wash,GRA P...",07051590488,3,...,2.500000,0,1.000000,NaN,2.500000,5.0,NaN,NaN,4.824801,6.988020
3,5555,Emmy Care Medical Centre,Medical services,Hospital,6.678829,3.461104,0,"NO. 119, IGBO OLOMU ROAD, AZO ROCK BUS STOP AG...","08109172595, 08060845319",4,...,2.500000,0,1.000000,NaN,2.500000,5.0,NaN,NaN,6.678829,3.461104
4,5555,Emmy Care Medical Centre,Medical services,Hospital,6.678829,3.461104,0,"NO. 119, IGBO OLOMU ROAD, AZO ROCK BUS STOP AG...","08109172595, 08060845319",4,...,2.500000,0,1.000000,NaN,2.500000,5.0,NaN,NaN,6.678829,3.461104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5584,37,Aniyun Hospital Ltd,Medical services,Hospital,6.553904,3.386898,0,"3, Femi Aderibigbe Close, Ifako, Gbagada, Lagos",", 08183696538, whatsapp: 09042839131",3,...,2.500000,1,0.041667,1.0,0.104168,5.0,NaN,NaN,6.553904,3.386898
5585,3574,Jubilee Hospital,Medical services,Hospital,7.429803,3.896045,0,"No 1, Samonda Business Complex, Behind Trade F...","09043102662, 08023254286, 09026655788",4,...,2.500000,1,1.000000,NaN,2.500000,5.0,NaN,NaN,7.429803,3.896045
5586,1012,Brighthouse Optical Clinic,Optical services,Optical Center,6.543965,3.323177,0,"1 Chivita Avenue,By 2/4 Jay Momoh Street,Ajao ...","08173965673, 08053796150, 07067539570",3,...,2.500000,0,1.000000,NaN,2.500000,5.0,NaN,NaN,6.543965,3.323177
5587,1581,Ethel Dental Clinic,Dental services,Dental Clinic,4.828084,7.062785,0,18 Trans woji road Elelenwo,08032014035,3,...,3.333333,1,1.000000,NaN,3.333333,5.0,NaN,NaN,4.828084,7.062785


In [9]:
def merge_columns(row):
    return ' '.join(str(cell) for cell in row if not pd.isna(cell))
NPS_DATA['CUSTOMER_RESPONSE'] = NPS_DATA[['we_are_sorry_that_you_had_an_unpleasant_experience._can_you_please_provide_more_information_on_why_you_gave_that_rating?',
                                          'we_are_glad_you_had_a_great_experience._can_you_please_provide_more_information_on_why_you_gave_that_rating?',
                                          'thank_you_for_your_response._can_you_please_provide_more_information_on_why_you_gave_that_rating?']].apply(merge_columns, axis=1)
NPS_DATA.drop(columns=['we_are_sorry_that_you_had_an_unpleasant_experience._can_you_please_provide_more_information_on_why_you_gave_that_rating?',
                                          'we_are_glad_you_had_a_great_experience._can_you_please_provide_more_information_on_why_you_gave_that_rating?',
                                          'thank_you_for_your_response._can_you_please_provide_more_information_on_why_you_gave_that_rating?'], inplace=True)
NPS_DATA_COI = NPS_DATA[['CUSTOMER_RESPONSE', 'provider_name']]

In [10]:
PROVIDER_RANKING_NPS

,provider_id,provider_name,type_service,type_service_name,provider_latitude,provider_longitude,is_internal_provider,address,telephone,provider_tier,...,raw_cost_star_rating,has_cost_stars,fwa_multiplier,months_since_last_fraud,cost_star_rating,clinical_stars,months_since_last_morbidity_incident,months_since_last_mortality_incident,latitude,longitude
0,5556,Bioflourishing Hospital,Medical services,Hospital,6.516740,3.307932,0,"63 BAYO OYEWALE,AGO PALACE WAY OKOTA","08037016681, 09018092810",4,...,2.500000,0,1.000000,NaN,2.500000,5.0,NaN,NaN,6.516740,3.307932
1,5554,Hephzibah Eye Care - Port Harcourt,Optical services,Optical Center,4.824801,6.988020,0,"Tombia Street, opposite Mandela Car Wash,GRA P...",07051590488,3,...,2.500000,0,1.000000,NaN,2.500000,5.0,NaN,NaN,4.824801,6.988020
2,5554,Hephzibah Eye Care - Port Harcourt,Optical services,Optical Center,4.824801,6.988020,0,"Tombia Street, opposite Mandela Car Wash,GRA P...",07051590488,3,...,2.500000,0,1.000000,NaN,2.500000,5.0,NaN,NaN,4.824801,6.988020
3,5555,Emmy Care Medical Centre,Medical services,Hospital,6.678829,3.461104,0,"NO. 119, IGBO OLOMU ROAD, AZO ROCK BUS STOP AG...","08109172595, 08060845319",4,...,2.500000,0,1.000000,NaN,2.500000,5.0,NaN,NaN,6.678829,3.461104
4,5555,Emmy Care Medical Centre,Medical services,Hospital,6.678829,3.461104,0,"NO. 119, IGBO OLOMU ROAD, AZO ROCK BUS STOP AG...","08109172595, 08060845319",4,...,2.500000,0,1.000000,NaN,2.500000,5.0,NaN,NaN,6.678829,3.461104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5584,37,Aniyun Hospital Ltd,Medical services,Hospital,6.553904,3.386898,0,"3, Femi Aderibigbe Close, Ifako, Gbagada, Lagos",", 08183696538, whatsapp: 09042839131",3,...,2.500000,1,0.041667,1.0,0.104168,5.0,NaN,NaN,6.553904,3.386898
5585,3574,Jubilee Hospital,Medical services,Hospital,7.429803,3.896045,0,"No 1, Samonda Business Complex, Behind Trade F...","09043102662, 08023254286, 09026655788",4,...,2.500000,1,1.000000,NaN,2.500000,5.0,NaN,NaN,7.429803,3.896045
5586,1012,Brighthouse Optical Clinic,Optical services,Optical Center,6.543965,3.323177,0,"1 Chivita Avenue,By 2/4 Jay Momoh Street,Ajao ...","08173965673, 08053796150, 07067539570",3,...,2.500000,0,1.000000,NaN,2.500000,5.0,NaN,NaN,6.543965,3.323177
5587,1581,Ethel Dental Clinic,Dental services,Dental Clinic,4.828084,7.062785,0,18 Trans woji road Elelenwo,08032014035,3,...,3.333333,1,1.000000,NaN,3.333333,5.0,NaN,NaN,4.828084,7.062785


In [11]:
PROVIDER_RANKING_NPS_COI = PROVIDER_RANKING_NPS[['provider_name', 'type_service', 'address']]

In [12]:
NPS_DATA

,provider_type,encounter_date,provider_name,category,manual_categorization,manual_sub-category,facility_cleanliness,overall_wait_time,staff_attitude/conduct,consultation_quality,equipment_quality,medication_quality,approval_wait_time,overall_wait_time2,CUSTOMER_RESPONSE
0,1,01 Aug 2023,REHOBOTH SPECIALIST HOSPITAL,Passive,None,None,Satisfied,Neutral,Satisfied,Satisfied,Neutral,Satisfied,None,None,It took time before attending with my condition
1,1,01 Aug 2023,Faith Care Hospital,Detractor,Bad customer service,None,Satisfied,Dissatisfied,Neutral,Neutral,Neutral,Dissatisfied,None,None,"I visited on the 31st of July, and was told to..."
2,9,01 Aug 2023,THALIA BY ALICE,Promoter,Good customer service,Staff attitude,Satisfied,Satisfied,Satisfied,None,Satisfied,None,None,None,Very courteous staff
3,6,01 Aug 2023,Clina - Lancet Laboratories - Ikeja,Passive,None,None,Satisfied,Neutral,Neutral,Satisfied,Satisfied,Satisfied,None,None,Swift service
4,1,01 Aug 2023,GreenLife Hospital -Ilupeju,Promoter,Good customer service,Staff attitude,Satisfied,Satisfied,Satisfied,Satisfied,Neutral,Satisfied,None,None,They handle customer with care n love and quic...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22947,1,12 Jan 2024,White Dove Hospital,Detractor,None,None,Neutral,Satisfied,Dissatisfied,Neutral,Dissatisfied,Dissatisfied,Not Applicable,More than 30 mins but within 1 hour,It was not a pleasant experience. It took thre...
22948,6,31 Jan 2024,EL-LAB LABORATORIES LIMITED - FESTAC,Promoter,None,None,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,15 mins or less,More than 30 mins but within 1 hour,Great customer service
22949,1,23 Jan 2024,Peamak Hospital and Maternity Centre,Passive,None,None,None,None,None,None,None,None,None,None,They have well trained medical practitioners
22950,1,31 Jan 2024,Greenlands Specialist Hospital,Promoter,None,None,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,More than 15 mins but within 30 mins,More than 30 mins but within 1 hour,I like the Greenland for the good effort on me


In [47]:
PROVIDER_RANKING_NPS_COI

,provider_name,type_service,address
0,Bioflourishing Hospital,Medical services,"63 BAYO OYEWALE,AGO PALACE WAY OKOTA"
1,Hephzibah Eye Care - Port Harcourt,Optical services,"Tombia Street, opposite Mandela Car Wash,GRA Phase 3,Port Harcourt,Rivers State"
2,Hephzibah Eye Care - Port Harcourt,Optical services,"Tombia Street, opposite Mandela Car Wash,GRA Phase 3,Port Harcourt,Rivers State"
3,Emmy Care Medical Centre,Medical services,"NO. 119, IGBO OLOMU ROAD, AZO ROCK BUS STOP AGRIC, IKORODU,"
4,Emmy Care Medical Centre,Medical services,"NO. 119, IGBO OLOMU ROAD, AZO ROCK BUS STOP AGRIC, IKORODU,"
...,...,...,...
5584,Aniyun Hospital Ltd,Medical services,"3, Femi Aderibigbe Close, Ifako, Gbagada, Lagos"
5585,Jubilee Hospital,Medical services,"No 1, Samonda Business Complex, Behind Trade Fair Complex , Opp Ventura Mall, U.I - Sango Road, Ibadan , Oyo State"
5586,Brighthouse Optical Clinic,Optical services,"1 Chivita Avenue,By 2/4 Jay Momoh Street,Ajao Estate,Oshodi -Isolo,Lagos."
5587,Ethel Dental Clinic,Dental services,18 Trans woji road Elelenwo


In [13]:
NPS_DATA_COI

,CUSTOMER_RESPONSE,provider_name
0,It took time before attending with my condition,REHOBOTH SPECIALIST HOSPITAL
1,"I visited on the 31st of July, and was told to...",Faith Care Hospital
2,Very courteous staff,THALIA BY ALICE
3,Swift service,Clina - Lancet Laboratories - Ikeja
4,They handle customer with care n love and quic...,GreenLife Hospital -Ilupeju
...,...,...
22947,It was not a pleasant experience. It took thre...,White Dove Hospital
22948,Great customer service,EL-LAB LABORATORIES LIMITED - FESTAC
22949,They have well trained medical practitioners,Peamak Hospital and Maternity Centre
22950,I like the Greenland for the good effort on me,Greenlands Specialist Hospital


In [14]:
NPS_DATA_QA = pd.merge(PROVIDER_RANKING_NPS_COI, NPS_DATA_COI, left_on = 'provider_name', 
                       right_on = 'provider_name')
NPS_DATA_QA.drop_duplicates(inplace=True)

In [15]:
NPS_DATA_QA

,provider_name,type_service,address,CUSTOMER_RESPONSE
0,"O.L.A Catholic Hospital, Oluyoro",Medical services,"Oluyoro Oke-Ofa,P.O. Box 7044,Secretariat, Ibadan",They are friendly and attend to me well
1,"O.L.A Catholic Hospital, Oluyoro",Medical services,"Oluyoro Oke-Ofa,P.O. Box 7044,Secretariat, Ibadan","The reception was great, their services was aw..."
2,"O.L.A Catholic Hospital, Oluyoro",Medical services,"Oluyoro Oke-Ofa,P.O. Box 7044,Secretariat, Ibadan","They are good at their work, and how their pat..."
3,"O.L.A Catholic Hospital, Oluyoro",Medical services,"Oluyoro Oke-Ofa,P.O. Box 7044,Secretariat, Ibadan",They're good at what they do
12,Luglow Aesthetics Spa Magodo,Spa services,"24C, Abdul Quadri Adebiyi Street, Magodo Phase II",Twas okay
...,...,...,...,...
27890,Aniyun Hospital Ltd,Medical services,"3, Femi Aderibigbe Close, Ifako, Gbagada, Lagos",There was no restriction whatsoever in attendi...
27891,Aniyun Hospital Ltd,Medical services,"3, Femi Aderibigbe Close, Ifako, Gbagada, Lagos",They waste a lot of time
27925,Le Lieu De Leilas,Spa services,"44 Okeho Street, Ire-Akari, Isolo, Lagos.",I enjoyed my spa session
27926,Le Lieu De Leilas,Spa services,"44 Okeho Street, Ire-Akari, Isolo, Lagos.",They treat their client nicely and give them t...


In [16]:
NPS_DATA_QA.shape

(10255, 4)

In [25]:
NPS_DATA_SAMPLE = NPS_DATA_QA.sample(frac=0.1, replace=False, random_state=1)
#NPS_DATA_SAMPLE = NPS_DATA_QA

In [26]:
NPS_DATA_SAMPLE.shape

(1026, 4)

In [27]:
NPS_DATA_SAMPLE

,provider_name,type_service,address,CUSTOMER_RESPONSE
7063,Pison Hospital,Medical services,"3535, GADO NASCO ROAD, BY MTN JUNCTION, PHASE ...",good
18591,R-Jolad Hospital Nig. Ltd,Medical services,"1, Akinyede Street, By New Garage Bariga, Gbag...","Very terrible , some of the staffs are time wa..."
18510,R-Jolad Hospital Nig. Ltd,Medical services,"1, Akinyede Street, By New Garage Bariga, Gbag...",Slow to respond n not polite
8343,Kingtrust Medical Center,Medical services,"105, Ogunlana Drive, Surulere lagos.",It was a good experience
1928,Reliance Family Clinic - Port Harcourt,Medical services,"No. 3 Rumuola link Road, opposite women of fai...",Delay in attending to me due to many people.\n...
...,...,...,...,...
1242,Unity Medical Centre,Medical services,"31 Sango Eleyele Road, Eleyele Ibadan",The service is OK.
22588,Longing Medical Centre,Medical services,"1, Josepha Close off Oguntana Ogundeji Street,...",There friendly and caring experience d
12383,Jefis Specialist Hospital Ltd,Medical services,"9, Adegbola Street, Off Baale Animashaun Road,...",The doctors midwives and nurses attended did well
14858,Broad Places Radiology,Medical Diagnostic services,15 BABATUNDE STREET OFF OGUNLANA DRIVE SURULER...,The response was quick


In [28]:
NPS_LABELS = ['lack of vitamin c in customer plan benefits',
              'lack of Paediatric care in customer plan benefits',
              'Lack of medical benefits in customer plan',
              'lack of typhoid drugs',

              'attitude of nurses',
              'attitude of doctors',
              'cleanliness of hosptials',
              'medication quality',
              'Front desk related',
              'attitude of receptionist',
              'place hygeine',
              'quality of building',
              'cleanliness',
              'customer visiting the hospital',
              'clinic related',
              'lack of female coworkers',

              'call center agents',
              'response to phone calls',
              'phone calls related',

              'waiting for doctors',
              'long queues in hospital',
              'delaying in getting test results',

              'Unavailable drug for pick-up at pharmacy',
              'medication pickup related',

              'customer confusion about the benefits',
              'lack of communication on changing providers on plan',

              'hospitals near the customer city',

              'internet work issue',
              'app dysfunction',
            
              'waiting time',
              'time related',
              'access related',
              'doctor related',
              'nurse related',
              'clinic related',
              'delivery related',
              'customer service',
              'delay',
              'medication, treatment, or drug related'
             ]          
              

In [29]:
len(NPS_DATA_SAMPLE)

1026

In [30]:
if not NPS_LABELS:
    raise ValueError("new_labels is empty. You must include at least one label.")

detailed_scores = []
counter = 0

for i in range(len(NPS_DATA_SAMPLE)):
    input_text = NPS_DATA_SAMPLE['CUSTOMER_RESPONSE'].iloc[i]

    if not input_text.strip():
        print(f"Skipping empty or whitespace 'CUSTOMER_RESPONSE' at index {i}.")
        continue

    try:
        model_dict = classifier(input_text, NPS_LABELS, multi_label=True)
        result_dict = dict(zip(model_dict.get('labels'), model_dict.get('scores')))
        score_dict = {'CUSTOMER_RESPONSE': input_text}  # using input_text directly
        score_dict.update(result_dict)
        detailed_scores.append(score_dict)
        print('The step is', counter)
        

        # print("Input Text for the model: ", input_text)
        # for label in NPS_LABELS:
        #     print(f"Confidence Score for {label}: ", result_dict.get(label, "N/A"))  # Default to "N/A" if label not found
        # print('\n\n')
    except Exception as e:
        # Log the exception and skip this iteration
        print(f"An error occurred at index {i} with text: {input_text}. Error: {e}")
        continue
    counter = counter +1

detailed_scores_df = pd.DataFrame(detailed_scores)

The step is 0
The step is 1
The step is 2
The step is 3
The step is 4
The step is 5
The step is 6
The step is 7
The step is 8
The step is 9
The step is 10
The step is 11
The step is 12
The step is 13
The step is 14
The step is 15
The step is 16
The step is 17
The step is 18
The step is 19
The step is 20
The step is 21
The step is 22
The step is 23
The step is 24
The step is 25
The step is 26
The step is 27
The step is 28
The step is 29
The step is 30
The step is 31
The step is 32
The step is 33
The step is 34
The step is 35
The step is 36
The step is 37
The step is 38
The step is 39
The step is 40
The step is 41
The step is 42
The step is 43
The step is 44
The step is 45
The step is 46
The step is 47
The step is 48
The step is 49
The step is 50
The step is 51
The step is 52
The step is 53
The step is 54
The step is 55
The step is 56
The step is 57
The step is 58
The step is 59
The step is 60
The step is 61
The step is 62
The step is 63
The step is 64
The step is 65
The step is 66
The s

In [33]:
PreClassified = NPS_DATA_SAMPLE[['type_service', 'CUSTOMER_RESPONSE', 'provider_name']]

In [34]:
detailed_scores_df['Health_benefits_coverage'] = ((detailed_scores_df['lack of vitamin c in customer plan benefits']>0.7).astype(int)|
                                        (detailed_scores_df['lack of Paediatric care in customer plan benefits']>0.9).astype(int)|
                                        (detailed_scores_df['Lack of medical benefits in customer plan']>0.9).astype(int)
                                                  | (detailed_scores_df['lack of typhoid drugs']>0.9).astype(int)
                                        )

detailed_scores_df['provider_quality'] = ((detailed_scores_df['attitude of nurses'] > 0.95).astype(int)|
                                               (detailed_scores_df['attitude of doctors']>0.8).astype(int)|
                                            (detailed_scores_df['cleanliness of hosptials'] > 0.95).astype(int)|
                                     (detailed_scores_df['medication quality'] > 0.95).astype(int)|
                                          (detailed_scores_df['Front desk related'] > 0.95).astype(int)|
                                     (detailed_scores_df['attitude of receptionist'] > 0.95).astype(int)|
                                     # (detailed_scores_df['surgeon related'] > 0.95).astype(int)|
                                     (detailed_scores_df['place hygeine'] > 0.95).astype(int)|
                                     (detailed_scores_df['quality of building'] > 0.95).astype(int)|
                                     (detailed_scores_df['cleanliness'] > 0.99).astype(int)|
                                     (detailed_scores_df['customer visiting the hospital'] > 0.95).astype(int)|
                                     (detailed_scores_df['clinic related'] > 0.95).astype(int)
                                          |(detailed_scores_df['lack of female coworkers'] > 0.95).astype(int)
                                         )
                                         

detailed_scores_df['RCC_quality'] = ((detailed_scores_df['call center agents']>0.9).astype(int)|
                                        (detailed_scores_df['response to phone calls']>0.95).astype(int)|
                                        (detailed_scores_df['phone calls related']>0.9).astype(int))

detailed_scores_df['provider_wait_times'] = ((detailed_scores_df['waiting for doctors'] > 0.9).astype(int) |
                                                (detailed_scores_df['long queues in hospital'] > 0.9).astype(int)|
                                             #  (detailed_scores_df['Authorization code'] > 0.9).astype(int)|
                                               (detailed_scores_df['delaying in getting test results'] > 0.99).astype(int))

detailed_scores_df['medication_related'] = ((detailed_scores_df['Unavailable drug for pick-up at pharmacy']>0.9).astype(int)|
                                        (detailed_scores_df['medication pickup related']>0.9).astype(int))

detailed_scores_df['customer_education'] = ( (detailed_scores_df['customer confusion about the benefits']>0.95).astype(int)|
                                              (detailed_scores_df['lack of communication on changing providers on plan']>0.95).astype(int))

detailed_scores_df['provider_network'] =  (detailed_scores_df['hospitals near the customer city']>0.95).astype(int)

detailed_scores_df['tech_issues'] = ( (detailed_scores_df['internet work issue']>0.95).astype(int)|
                                              (detailed_scores_df['app dysfunction']>0.95).astype(int))

In [35]:
detailed_scores_df['wait_time_related'] = (detailed_scores_df['waiting time'] > 0.99).astype(int)
detailed_scores_df['time related'] = (detailed_scores_df['time related'] > 0.99).astype(int)
detailed_scores_df['access related'] = (detailed_scores_df['access related'] > 0.99).astype(int)
detailed_scores_df['doctor related'] = (detailed_scores_df['doctor related'] > 0.99).astype(int)
detailed_scores_df['nurse related'] = (detailed_scores_df['nurse related'] > 0.99).astype(int)
detailed_scores_df['clinic related'] = (detailed_scores_df['clinic related'] > 0.99).astype(int)
detailed_scores_df['delivery related'] = (detailed_scores_df['delivery related'] > 0.98).astype(int)
detailed_scores_df['customer service related'] = (detailed_scores_df['customer service'] > 0.99).astype(int)
detailed_scores_df['delay'] = (detailed_scores_df['delay'] > 0.99).astype(int)
detailed_scores_df["medication, treatment, or drug related"] = (detailed_scores_df["medication, treatment, or drug related"] > 0.9).astype(int)

# If the customer response got classified as ("medication, drug, treatment" or medication relaed) related and it's not related to delivery --> then it's related to medicatuion quality aka provider provider.
detailed_scores_df.loc[(detailed_scores_df['medication, treatment, or drug related'] == 1) & (detailed_scores_df['delivery related'] == 0), 'provider_quality'] = 1
detailed_scores_df.loc[(detailed_scores_df['medication_related'] == 1) & (detailed_scores_df['delivery related'] == 0), 'provider_quality'] = 1

# if the customer response contains the word "customer service", then it's RCC related
detailed_scores_df.loc[detailed_scores_df['CUSTOMER_RESPONSE'].str.contains("customer service", case=False, na=False), 'RCC_quality'] = 1

# If the customer response contains "mosquitoes" then it's related to equipment, which means it's related to provider quality.
detailed_scores_df.loc[detailed_scores_df['CUSTOMER_RESPONSE'].str.contains("mosquitoes", case=False, na=False), 'provider_quality'] = 1

# if the customer response contains "during my visit", then it means that the customer went there to hospital, clinic..etc. which means provider quality not RCC
detailed_scores_df.loc[detailed_scores_df['CUSTOMER_RESPONSE'].str.contains("during my visit", case=False, na=False), 'provider_quality'] = 1
detailed_scores_df.loc[detailed_scores_df['CUSTOMER_RESPONSE'].str.contains("during my visit", case=False, na=False), 'RCC_quality'] = 0

# If the customer response got classified as doctor related, then it's a provider_quality not RCC
detailed_scores_df.loc[detailed_scores_df['doctor related'] == 1, 'provider_quality'] = 1
#detailed_scores_df.loc[detailed_scores_df['nurse related'] == 1, 'provider_quality'] = 1

# If the customer response contains "gym" then provider_quality=1
detailed_scores_df.loc[detailed_scores_df['CUSTOMER_RESPONSE'].str.contains("gym", case=False, na=False), 'provider_quality'] = 1
detailed_scores_df.loc[detailed_scores_df['CUSTOMER_RESPONSE'].str.contains("gym", case=False, na=False), 'RCC_quality'] = 0

# If the customer_response got classified as "is access" and "time related" then it's gonna be provider_wait_time
detailed_scores_df.loc[(detailed_scores_df['access related'] == 1) & (detailed_scores_df['time related'] == 1), 'provider_wait_times'] = 1

# if it's delay related and RCC_quality = 0, then it's a provider wait time.
detailed_scores_df.loc[(detailed_scores_df['delay'] == 1) & (detailed_scores_df['RCC_quality'] == 0), 'provider_wait_times'] = 1


In [36]:
detailed_scores_df['Medical_Staff'] =  ((detailed_scores_df['attitude of doctors']>0.9).astype(int)|
                                        (detailed_scores_df['attitude of nurses']>0.9).astype(int)|
                                       (detailed_scores_df['doctor related']==1).astype(int))
                                       # |(detailed_scores_df['lack of female coworkers']>0.9).astype(int))

detailed_scores_df.loc[detailed_scores_df['CUSTOMER_RESPONSE'].str.contains("doctors", case=False, na=False), 'Medical_Staff'] = 1
detailed_scores_df.loc[detailed_scores_df['CUSTOMER_RESPONSE'].str.contains("nurse", case=False, na=False), 'Medical_Staff'] = 1
detailed_scores_df.loc[detailed_scores_df['CUSTOMER_RESPONSE'].str.contains("dentist", case=False, na=False), 'Medical_Staff'] = 1
detailed_scores_df.loc[detailed_scores_df['CUSTOMER_RESPONSE'].str.contains("staff", case=False, na=False), 'Medical_Staff'] = 1
detailed_scores_df.loc[detailed_scores_df['CUSTOMER_RESPONSE'].str.contains("massage", case=False, na=False), 'Medical_Staff'] = 1

detailed_scores_df['Receptionist_Front_Desk'] = ((detailed_scores_df['attitude of receptionist']>0.9).astype(int)|
                                        (detailed_scores_df['Front desk related']>0.95).astype(int))

detailed_scores_df['Facility Quality'] = ((detailed_scores_df['quality of building']>0.98).astype(int)|
                                        (detailed_scores_df['place hygeine']>0.98).astype(int)|
                                         (detailed_scores_df['cleanliness of hosptials']>0.98).astype(int))
detailed_scores_df.loc[detailed_scores_df['CUSTOMER_RESPONSE'].str.contains("ambience", case=False, na=False), 'Facility Quality'] = 1
detailed_scores_df.loc[detailed_scores_df['CUSTOMER_RESPONSE'].str.contains("welcome environment", case=False, na=False), 'Facility Quality'] = 1
detailed_scores_df.loc[detailed_scores_df['CUSTOMER_RESPONSE'].str.contains("clean", case=False, na=False), 'Facility Quality'] = 1
detailed_scores_df.loc[detailed_scores_df['CUSTOMER_RESPONSE'].str.contains("facilities", case=False, na=False), 'Facility Quality'] = 1
detailed_scores_df.loc[detailed_scores_df['CUSTOMER_RESPONSE'].str.contains("empathy", case=False, na=False), 'Facility Quality'] = 1
detailed_scores_df.loc[detailed_scores_df['CUSTOMER_RESPONSE'].str.contains("hospital", case=False, na=False), 'Facility Quality'] = 1


detailed_scores_df['Medication_Quality'] = ((detailed_scores_df['medication quality']>0.95).astype(int))
detailed_scores_df.loc[detailed_scores_df['CUSTOMER_RESPONSE'].str.contains("treatment", case=False, na=False), 'Medication_Quality'] = 1

detailed_scores_df['provider_wait_times'] = ((detailed_scores_df['time related']==1).astype(int) & (detailed_scores_df['medication_related']==1).astype(int) )
detailed_scores_df.loc[detailed_scores_df['CUSTOMER_RESPONSE'].str.contains("delay", case=False, na=False), 'provider_wait_times'] = 1
detailed_scores_df.loc[detailed_scores_df['CUSTOMER_RESPONSE'].str.contains("computerized process", case=False, na=False), 'provider_wait_times'] = 1
detailed_scores_df.loc[detailed_scores_df['CUSTOMER_RESPONSE'].str.contains("code", case=False, na=False), 'provider_wait_times'] = 1
detailed_scores_df.loc[detailed_scores_df['CUSTOMER_RESPONSE'].str.contains("approved", case=False, na=False), 'provider_wait_times'] = 1
detailed_scores_df.loc[detailed_scores_df['CUSTOMER_RESPONSE'].str.contains("Timelineness", case=False, na=False), 'provider_wait_times'] = 1

detailed_scores_df.loc[detailed_scores_df['CUSTOMER_RESPONSE'].str.contains("lack", case=False, na=False), 'Health_benefits_coverage'] = 1


In [37]:
NPS_DATA_SAMPLE

,provider_name,type_service,address,CUSTOMER_RESPONSE
7063,Pison Hospital,Medical services,"3535, GADO NASCO ROAD, BY MTN JUNCTION, PHASE ...",good
18591,R-Jolad Hospital Nig. Ltd,Medical services,"1, Akinyede Street, By New Garage Bariga, Gbag...","Very terrible , some of the staffs are time wa..."
18510,R-Jolad Hospital Nig. Ltd,Medical services,"1, Akinyede Street, By New Garage Bariga, Gbag...",Slow to respond n not polite
8343,Kingtrust Medical Center,Medical services,"105, Ogunlana Drive, Surulere lagos.",It was a good experience
1928,Reliance Family Clinic - Port Harcourt,Medical services,"No. 3 Rumuola link Road, opposite women of fai...",Delay in attending to me due to many people.\n...
...,...,...,...,...
1242,Unity Medical Centre,Medical services,"31 Sango Eleyele Road, Eleyele Ibadan",The service is OK.
22588,Longing Medical Centre,Medical services,"1, Josepha Close off Oguntana Ogundeji Street,...",There friendly and caring experience d
12383,Jefis Specialist Hospital Ltd,Medical services,"9, Adegbola Street, Off Baale Animashaun Road,...",The doctors midwives and nurses attended did well
14858,Broad Places Radiology,Medical Diagnostic services,15 BABATUNDE STREET OFF OGUNLANA DRIVE SURULER...,The response was quick


In [39]:
QA_DATA = detailed_scores_df[['CUSTOMER_RESPONSE','Health_benefits_coverage', 'provider_quality', 'RCC_quality',
       'provider_wait_times','customer_education',
       'provider_network', 'tech_issues', 'Medical_Staff', 'Receptionist_Front_Desk', 'Facility Quality', 'Medication_Quality', 'medication, treatment, or drug related']]

NPS_DATA_SAMPLE = NPS_DATA_SAMPLE[['CUSTOMER_RESPONSE', 'type_service', 'provider_name']]

NPS_DATA_QA = pd.merge(NPS_DATA_SAMPLE, QA_DATA, 
                       left_on = 'CUSTOMER_RESPONSE', 
                       right_on = 'CUSTOMER_RESPONSE')

In [40]:
check_columns = ['Medical_Staff', 'Receptionist_Front_Desk', 'Facility Quality', 'Medication_Quality']
NPS_DATA_QA['provider_quality'] = NPS_DATA_QA[check_columns].apply(lambda x: 1 if 1 in x.values else 0, axis=1)

In [43]:
NPS_DATA_QA['provider_quality'] = NPS_DATA_QA['provider_name'].str.contains('Reliance|Reliance Family').astype(int)
NPS_DATA_QA['provider_quality'] = (~NPS_DATA_QA['provider_name'].str.contains('Reliance|Reliance Family')).astype(int)
NPS_DATA_QA['RCC_quality'] = 1 - NPS_DATA_QA['provider_quality']


In [44]:
NPS_DATA_QA

,CUSTOMER_RESPONSE,type_service,provider_name,Health_benefits_coverage,provider_quality,RCC_quality,provider_wait_times,customer_education,provider_network,tech_issues,Medical_Staff,Receptionist_Front_Desk,Facility Quality,Medication_Quality,"medication, treatment, or drug related"
0,good,Medical services,Pison Hospital,0,1,0,0,0,0,0,0,0,0,0,0
1,good,Medical services,Pison Hospital,0,1,0,0,0,0,0,0,0,0,0,0
2,good,Medical services,Micel Hospital,0,1,0,0,0,0,0,0,0,0,0,0
3,good,Medical services,Micel Hospital,0,1,0,0,0,0,0,0,0,0,0,0
4,"Very terrible , some of the staffs are time wa...",Medical services,R-Jolad Hospital Nig. Ltd,0,1,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170,The service is OK.,Medical services,Unity Medical Centre,0,1,0,0,0,0,0,0,0,0,0,0
1171,There friendly and caring experience d,Medical services,Longing Medical Centre,0,1,0,0,0,0,0,0,0,0,0,0
1172,The doctors midwives and nurses attended did well,Medical services,Jefis Specialist Hospital Ltd,0,1,0,0,0,0,0,1,0,0,0,0
1173,The response was quick,Medical Diagnostic services,Broad Places Radiology,0,1,0,0,0,0,0,0,0,0,0,0


In [ ]:
# 102 --> right
# 101 --> right
# 100 --> right
# 99 --> right
# 98 --> I believe it's either right or RCC, Not sure! 
# 97 --> right
# 96 --> right
# 95 --> wrong, added facilites in str.location
# 94 --------------------------------------------> Think about it and discuss with Jacob, right! 
# 93 -----> right
# 92 ---> right
# 91 --> right
# 90 --> right
# 89 --> wrong, added "delay" in provider wait times.
# 88 ---------------> discussable, but I believe it's right! 
# 87 --> right
# 86 --> wrong! added a lack of female coworkers
# 85 --> right
# 84 --> right
# 83 --> right
# 82 --> got it wrong, added computerized process in the provider wait times.
# 81 --> right
# 80 --> Make sure to discuss this with Jacob, my idea is about making "response" and "slow" in string containers then put them in the RCC
# 79 --> right
# 78 --> right
# 77 --> added "lack" as it should be added under the health benefits coverage! discuss it with Jacob too
# 76 --> right
# 75 --> right
# 74 --> right
# 73 --> right
# 72 --> right
# 71 --> right
# 70 --> wrong, added "approved" into the string container then to provider wait timers
# 69 --> right
# 68 --> right
# 67 --> right, but I believe it's still generic
# 66 --> right
# 65 --> right
# 64 --> right
# 63 --> right
# 62 --> right 
# 61 --> right
# 60 --> wrong, add "lack of typhoid drugs" and it shouldn't be classified as "medical staff" 
# 59 --> right
# 58 --> right
# 57 --> right, but make sure to differentiate between RCC and Provider Quality
# 56 --> right
# 55 --> right
# 54 --> right
# 53 --> right
# 52 --> wrong, add "Is_provider then Is_Time" to handle this case! 
# 51 --> right
# 50 --> right

In [ ]:
# no.17 --> how to solve it? 
# no.18 --> It's wrong, but what umbrealla should "lack of drugs" should be related to? 
# no.25 --> why there are classifying it into drugs related?  --> you should tune this columns! 
# 

In [45]:
QA_df = NPS_DATA_QA

In [46]:
QA_df

,CUSTOMER_RESPONSE,type_service,provider_name,Health_benefits_coverage,provider_quality,RCC_quality,provider_wait_times,customer_education,provider_network,tech_issues,Medical_Staff,Receptionist_Front_Desk,Facility Quality,Medication_Quality,"medication, treatment, or drug related"
0,good,Medical services,Pison Hospital,0,1,0,0,0,0,0,0,0,0,0,0
1,good,Medical services,Pison Hospital,0,1,0,0,0,0,0,0,0,0,0,0
2,good,Medical services,Micel Hospital,0,1,0,0,0,0,0,0,0,0,0,0
3,good,Medical services,Micel Hospital,0,1,0,0,0,0,0,0,0,0,0,0
4,"Very terrible , some of the staffs are time wa...",Medical services,R-Jolad Hospital Nig. Ltd,0,1,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170,The service is OK.,Medical services,Unity Medical Centre,0,1,0,0,0,0,0,0,0,0,0,0
1171,There friendly and caring experience d,Medical services,Longing Medical Centre,0,1,0,0,0,0,0,0,0,0,0,0
1172,The doctors midwives and nurses attended did well,Medical services,Jefis Specialist Hospital Ltd,0,1,0,0,0,0,0,1,0,0,0,0
1173,The response was quick,Medical Diagnostic services,Broad Places Radiology,0,1,0,0,0,0,0,0,0,0,0,0


In [47]:
def find_columns_with_one(row):
    cols = row.index[row == 1].tolist()
    return ', '.join(cols)

QA_df['main_topic'] = QA_df[['Health_benefits_coverage', 'provider_quality', 'RCC_quality', 'provider_wait_times', 'customer_education', 'provider_network', 'tech_issues']].apply(find_columns_with_one, axis=1)
QA_df['sub_topic'] = QA_df[['Medical_Staff', 'Receptionist_Front_Desk', 'Facility Quality', 'Medication_Quality']].apply(find_columns_with_one, axis=1)

In [48]:
#QA_df['sub_topic'] = QA_df['TYPE_SERVICE'] + ', '+ QA_df['sub_topic']

In [49]:
QA_df

,CUSTOMER_RESPONSE,type_service,provider_name,Health_benefits_coverage,provider_quality,RCC_quality,provider_wait_times,customer_education,provider_network,tech_issues,Medical_Staff,Receptionist_Front_Desk,Facility Quality,Medication_Quality,"medication, treatment, or drug related",main_topic,sub_topic
0,good,Medical services,Pison Hospital,0,1,0,0,0,0,0,0,0,0,0,0,provider_quality,
1,good,Medical services,Pison Hospital,0,1,0,0,0,0,0,0,0,0,0,0,provider_quality,
2,good,Medical services,Micel Hospital,0,1,0,0,0,0,0,0,0,0,0,0,provider_quality,
3,good,Medical services,Micel Hospital,0,1,0,0,0,0,0,0,0,0,0,0,provider_quality,
4,"Very terrible , some of the staffs are time wa...",Medical services,R-Jolad Hospital Nig. Ltd,0,1,0,0,0,0,0,1,0,0,0,1,provider_quality,Medical_Staff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170,The service is OK.,Medical services,Unity Medical Centre,0,1,0,0,0,0,0,0,0,0,0,0,provider_quality,
1171,There friendly and caring experience d,Medical services,Longing Medical Centre,0,1,0,0,0,0,0,0,0,0,0,0,provider_quality,
1172,The doctors midwives and nurses attended did well,Medical services,Jefis Specialist Hospital Ltd,0,1,0,0,0,0,0,1,0,0,0,0,provider_quality,Medical_Staff
1173,The response was quick,Medical Diagnostic services,Broad Places Radiology,0,1,0,0,0,0,0,0,0,0,0,0,provider_quality,


In [51]:
FINAL_df = QA_df[['CUSTOMER_RESPONSE', 'main_topic', 'sub_topic', 'type_service']]

In [52]:
QA_df['main_topic'].unique()

array(['provider_quality', 'RCC_quality, provider_wait_times',
       'RCC_quality', 'provider_quality, provider_wait_times',
       'Health_benefits_coverage, provider_quality',
       'provider_quality, provider_network',
       'provider_quality, customer_education, provider_network',
       'provider_quality, customer_education',
       'Health_benefits_coverage, provider_quality, customer_education',
       'Health_benefits_coverage, RCC_quality, customer_education',
       'provider_quality, tech_issues',
       'provider_quality, provider_wait_times, customer_education',
       'Health_benefits_coverage, provider_quality, provider_wait_times',
       'Health_benefits_coverage, provider_quality, provider_wait_times, customer_education',
       'RCC_quality, customer_education'], dtype=object)

In [53]:
FINAL_df.loc[1,:]

CUSTOMER_RESPONSE                good
main_topic           provider_quality
sub_topic                            
type_service         Medical services
Name: 1, dtype: object

In [54]:
FINAL_df['sub_topic'] = FINAL_df['sub_topic'].replace('', 'generic')


/var/folders/k_/pdztq4ls169gjt6b3pn7yx640000gn/T/ipykernel_13202/1163392326.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FINAL_df['sub_topic'] = FINAL_df['sub_topic'].replace('', 'generic')


In [56]:
FINAL_df

,CUSTOMER_RESPONSE,main_topic,sub_topic,type_service
0,good,provider_quality,generic,Medical services
1,good,provider_quality,generic,Medical services
2,good,provider_quality,generic,Medical services
3,good,provider_quality,generic,Medical services
4,"Very terrible , some of the staffs are time wa...",provider_quality,Medical_Staff,Medical services
...,...,...,...,...
1170,The service is OK.,provider_quality,generic,Medical services
1171,There friendly and caring experience d,provider_quality,generic,Medical services
1172,The doctors midwives and nurses attended did well,provider_quality,Medical_Staff,Medical services
1173,The response was quick,provider_quality,generic,Medical Diagnostic services


In [55]:
FINAL_df.to_csv('FINAL_df.csv')

In [59]:
upload_large_table(connection,FINAL_df,'NPS_QA_OP',if_exists='replace')

/Users/abdulrahmanmohsen/opt/miniconda3/envs/ml/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/Users/abdulrahmanmohsen/Documents/GitHub/data_science_utils/snowflake.py:196: UserWarning: The provided table name 'NPS_QA_OP' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  iter_df.to_sql(table_name,con=con,index=index,if_exists=if_exists,schema=schema,dtype=dtype)
